# Robots do it again!

dr. Vasilescu calls her MARS-O-HELP robot to do its magic once more: The robot is capable of controlling its kinematics pretty reliably, so what if she makes it hold the antenna towards a communication satellite and follow its trajectory?

<img src="../artwork/stranded/robot_holding_satellite_dish.png" width=60% />

## First task
Using the code you constructed for the simulation in the [theory notebook for trajectories](../theory/lab06_Trajectories.ipynb), implement a line trajectory for the AL5D robot for the following starting and ending poses:
    
|Coord|Initial|Final|
|-|-|-|
|<td colspan=3>Position<td colspan=2>
|X|0.16|0.18|
|Y|0.22|-0.18|
|Z|0.09|0.13|
|<td colspan=3>Orientation (Euler 'zyx' angles) <td colspan=2>
|roll|0|0|
|pitch|20|-90|
|yaw|0|10|
    
Then, execute the trajectory using the robot on your desk

In [ ]:
# You should only execute this cell once!
# If you execute it again, you need to restart the kernel and the robot
from al5d_control import *

# for sending the commands
rrob = AL5DControl()

In [ ]:
import time
# Solve the ikine for the respective interpolated poses
# The solution will be in radians!


# Take the robot in the configuration for the starting pose
rrob.robot_control(np.degrees(qs[0,0]),np.degrees(qs[0,1]),np.degrees(qs[0,2]),np.degrees(qs[0,3]),np.degrees(qs[0,4]),0)
time.sleep(3)

# Give degrees as input for the joint values
for i in range(steps):
    rrob.robot_control(np.degrees(qs[i,0]),np.degrees(qs[i,1]),np.degrees(qs[i,2]),np.degrees(qs[i,3]),np.degrees(qs[i,4]),0)
    ti.sleep(0.1)

## Second task

Calculate a series of poses so that the robot follows a trajectory of a circle on the XY plane, with radius $r=6cm$, and centre at $ X = 20cm$, $Y = 0cm$, and $Z = 4cm$. The end-effector should not be rotated w.r.t. the base frame (0 rotations). Then solve the inverse kinematics for those poses and send the resulting joint coordinates on the robot.

In [ ]:
from lab_functions import *
import swift

env = swift.Swift()
env.launch(realtime=True, browser='notebook')

al5d = rtb.models.URDF.AL5D_mdw()
al5d.q = [0,0,0,0,0]

arrived = False
env.add(al5d)